In [1]:
!pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 48.7 MB/s eta 0:00:00


In [2]:
from gurobipy import *

In [6]:
try:
    m = Model("Blending_Problem")

    gas_price = [70,60,50]
    crude_cost = [45,35,25]
    gas_j_demand = [3000,2000,1000]
    crude_avail = [5000,5000,5000]
    process_cost = 4
    demand_infla = 10
    capacity = 14000
    
    crude_octane_rating = [12,6,8]
    crude_sulfur_level = [0.005,0.02,0.03]
    gas_octane_rating = [10,8,6]
    gas_sulfur_level = [0.01,0.02,0.01]
    
    x_var_names = [["x"+str(i+1)+str(j+1) for i in range(len(crude_cost))] for j in range(len(gas_price))]
    a_var_names = ["a"+str(j+1) for j in range(len(gas_price))]
    
    x_var = []
    for i in range(len(crude_cost)):
        x_var.append([])
        for j in range(len(gas_price)):
            x_var[i].append(m.addVar(obj = gas_price[j] - crude_cost[i] - process_cost, vtype=GRB.CONTINUOUS, name="(%s)" % (x_var_names[i][j])))
    
    a_var = []
    for j in range(len(gas_price)):
        a_var.append(m.addVar(obj = -1, vtype=GRB.CONTINUOUS, name="(%s)" % (a_var_names[j])))
    
    m.modelSense = GRB.MAXIMIZE
    m.update()
    
    for j in range(len(gas_price)):
        m.addConstr(quicksum(x_var[i][j] for i in range(len(crude_cost))) == gas_j_demand[j] + demand_infla * a_var[j], "Gas %d requirement " % j)
        
    for i in range(len(crude_cost)):
        m.addConstr(quicksum(x_var[i][j] for j in range(len(gas_price))) <= crude_avail[i], "Crude Oil %d requirement " % i)
        
    m.addConstr(quicksum(x_var[i][j] for i in range(len(crude_cost)) for j in range(len(gas_price))) <= capacity, "capacity limit ")
    
    for j in range(len(gas_price)):
        m.addConstr(quicksum(x_var[i][j] * crude_octane_rating[i] for i in range(len(crude_cost))) >= gas_octane_rating[j] * quicksum(x_var[i][j] for i in range(len(crude_cost))))
        
    for j in range(len(gas_price)):
        m.addConstr(quicksum(x_var[i][j] * crude_sulfur_level[i] for i in range(len(crude_cost))) <= gas_sulfur_level[j] * quicksum(x_var[i][j] for i in range(len(crude_cost))))

    m.optimize()

    for v in m.getVars():
        print(v.varName, v.x)
            
    print("Obj:", m.objVal)

except GurobiError:
    print("Error reported")

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 13 rows, 12 columns and 45 nonzeros
Model fingerprint: 0x86e972fe
Coefficient statistics:
  Matrix range     [5e-03, 1e+01]
  Objective range  [1e+00, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 1e+04]
Presolve removed 4 rows and 3 columns
Presolve time: 0.01s
Presolved: 9 rows, 9 columns, 33 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.2410000e+05   3.094815e+03   0.000000e+00      0s
      11    2.8775000e+05   0.000000e+00   0.000000e+00      0s

Solved in 11 iterations and 0.02 seconds (0.00 work units)
Optimal objective  2.877500000e+05
(x11) 2000.0
(x21) 2333.3333333333335
(x31) 666.6666666666666
(x12) 1000.0
(x22) 3666.6666666666665
(x32) 333.3333333333333
(x13) 0.0
(x23) 3500.000000